<a href="https://colab.research.google.com/github/Atulkhiyani0909/ByteVerse_NayaSetu/blob/main/Chatbot_RAG/NyayaSetu_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

In [79]:
%pip install --quiet -U langchain langchain-community pymupdf langchain-huggingface langchain-pinecone pinecone-notebooks tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.2 MB/s eta 0:00:00


## Environment

In [39]:
from google.colab import userdata
import os

In [40]:
os.environ['PINECONE_API_KEY']=userdata.get('PINECONE_API_KEY')

In [41]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec


from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

from uuid import uuid4
import time

## Document Loading and Chunking

In [68]:
data_dir = '/content/Knowledge_base'
[print(i) for i in Path(data_dir).glob("*.pdf")]

/content/Knowledge_base/Corruption in Indian Railways_ Comprehensive Analy.pdf
/content/Knowledge_base/railway_protection_force_act_1957.pdf
/content/Knowledge_base/railways_act_1989.pdf
/content/Knowledge_base/Comprehensive Guide to Indian Railways Rules and R.pdf


[None, None, None, None]

In [83]:
# import re
from pathlib import Path
from pymupdf import Document
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# def extract_legal_sections(text: str) -> list:
#     """Enhanced extraction of legal references with subsections"""
#     pattern = r"""
#         (?i)                      # Case-insensitive
#         \b                        # Word boundary
#         (
#             Section|§|            # Section symbols
#             Article|Art\.|        # Article variations
#             Chapter|Ch\.|         # Chapter variations
#             Rule|Regulation|Reg\. # Rules/regulations
#             Clause|Subsection|    # Subsections
#             Schedule|Form|        # Schedules and forms
#             Part|Division|        # Document divisions
#             Act\sNo\.|            # Act numbers
#             [A-Z]{2,}             # All caps acronyms (e.g., RPF)
#         )
#         \s*                       # Optional whitespace
#         (
#             (?:[0-9]+[A-Z]*       # Base number with optional letter
#             (?:\([0-9A-Za-z]+\))* # Multiple nested subsections
#             (?:\.[0-9]+)*         # Decimal points
#             )|
#             [IVXLCDM]+            # Roman numerals
#         )
#         (?=\b|[,.;)])             # Lookahead for valid terminators
#     """
#     matches = re.findall(pattern, text, re.VERBOSE)
#     return [' '.join(match).strip() for match in matches if any(match)]

def process_pdf_document(file_path: str) -> list:
    """Process a single PDF document with enhanced metadata"""
    loader = PyMuPDFLoader(file_path)
    docs = loader.load()

    for doc in docs:
        # Enhanced metadata
        doc.metadata.update({
            "source": Path(file_path).name,
            # "sections": extract_legal_sections(doc.page_content),
        })

    return docs

def preprocess_legal_documents(data_dir: str = '/content/Knowledge_base') -> list:
    """Process all PDFs in directory with optimal chunking"""
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=300,
        chunk_overlap=100,
    )

    all_docs = []
    for pdf_file in Path(data_dir).glob('*.pdf'):
        print(f"Processing {pdf_file.name}...")
        docs = process_pdf_document(str(pdf_file))
        chunks = text_splitter.split_documents(docs)
        all_docs.extend(chunks)

    print(f"Processed {len(all_docs)} document chunks")
    return all_docs

# Usage
documents = preprocess_legal_documents()

Processing Corruption in Indian Railways_ Comprehensive Analy.pdf...
Processing railway_protection_force_act_1957.pdf...
Processing railways_act_1989.pdf...
Processing Comprehensive Guide to Indian Railways Rules and R.pdf...
Processed 328 document chunks


In [84]:
documents[:1]

[Document(metadata={'producer': 'Skia/PDF m127', 'creator': 'Chromium', 'creationdate': '2025-04-13T01:42:21+00:00', 'source': 'Corruption in Indian Railways_ Comprehensive Analy.pdf', 'file_path': '/content/Knowledge_base/Corruption in Indian Railways_ Comprehensive Analy.pdf', 'total_pages': 5, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-04-13T01:42:21+00:00', 'trapped': '', 'modDate': "D:20250413014221+00'00'", 'creationDate': "D:20250413014221+00'00'", 'page': 0}, page_content='This document provides an in-depth examination of corruption scenarios within Indian Railways,\nanalyzing their operational mechanics, legal implications, and actionable solutions for affected\npassengers and stakeholders. Drawing from recent cases, policy frameworks, and judicial\nprecedents, it serves as a comprehensive guide for navigating corruption-related challenges.\nScenario: Railway officials collude with contractors to manipulate tender processes,

## Embedding & Vector Store

In [85]:
index_name = 'nyayasetu'
pc = Pinecone() #setting up connection with PineCone

In [86]:
existing_index = [i.name for i in pc.list_indexes()] #checking for existing indexes
existing_index

['advanceragpipeline', 'graminsaathi', 'krishiarya']

In [87]:
if index_name not in existing_index:
    pc.create_index(
        name = index_name,
        dimension = 768,
        metric = 'cosine',
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)
else:
    print(f'Index {index_name} already exists')

In [88]:
# !pip install --upgrade numpy
# !pip install --upgrade scipy transformers
# This need to be done once as I am missing some packages

In [89]:
embeddings = HuggingFaceEmbeddings(
        model_name="nlpaueb/legal-bert-base-uncased",
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )

index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index,embedding=embeddings)

#testing first
# single = documents[:2]
# vector_store.add_documents(documents=single)

In [90]:
batch_size = 100

for i in range(0, len(documents), batch_size):
    batch_docs = documents[i:i+batch_size]
    batch_uuids = uuids[i:i+batch_size]

    try:
        vector_store.add_documents(documents=batch_docs, ids=batch_uuids)
        print(f"Batch {i//batch_size + 1} added successfully.")
    except Exception as e:
        print(f"Error adding batch {i//batch_size + 1}: {e}")


Batch 1 added successfully.
Batch 2 added successfully.
Batch 3 added successfully.
Batch 4 added successfully.


In [91]:
#Checking Retrival

retriever = vector_store.as_retriever(
    search_type = 'similarity_score_threshold',
    search_kwargs = {'k':5,'score_threshold':0.7},
)

In [92]:
retriever.get_relevant_documents('Fines in Railway')

[Document(id='de2da2c6-fb67-4b21-85e1-34ba54134df9', metadata={'author': '', 'creationDate': "D:20250413014221+00'00'", 'creationdate': '2025-04-13T01:42:21+00:00', 'creator': 'Chromium', 'file_path': '/content/Knowledge_base/Corruption in Indian Railways_ Comprehensive Analy.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20250413014221+00'00'", 'moddate': '2025-04-13T01:42:21+00:00', 'page': 2.0, 'producer': 'Skia/PDF m127', 'source': 'Corruption in Indian Railways_ Comprehensive Analy.pdf', 'subject': '', 'title': '', 'total_pages': 5.0, 'trapped': ''}, page_content='1. IRCTC’s Four-Pillar Strategy\n1. Prevention: Mandatory e-procurement portals with AI anomaly detection\n.\n[3]\n2. Detection: Bi-annual audits by KPMG-led consortium\n.\n[3]\n3. Response: 72-hour resolution timeline for vigilance complaints\n.\n[3]\n4. Improvement: ISO 37001 recertification every 18 months\n.\n[3]\n2. CBI’s Railway Corruption Cell\nSpecialized wing with 150 officers across zones\nCryptocurre